<a href="https://colab.research.google.com/github/its1ashish/langGraph/blob/main/Langgraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1. Install Dependencies

In [1]:
!pip install -qU google-generativeai==0.8.5 google-ai-generativelanguage==0.6.15 \
langgraph langchain langchain-google-genai openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.2/152.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 767.7/767.7 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 17.7 MB/s eta 0:00:00


Step 2. Imports and Secure API key Input

In [2]:
import os
import getpass
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage


In [3]:
os.environ['GOOGLE_API_KEY'] = getpass.getpass("Enter the Gemini API Key")

Enter the Gemini API Key··········


Step 3. Initialize Gemini 1.5 Flash

In [4]:
llm = ChatGoogleGenerativeAI(model="models/gemini-1.5-flash-latest", tempeature=0.2)

Step 4. Node to ask user for symptom

In [5]:
def get_symptom(state: dict)->dict:
  symptom=input("Please enter your symptom")
  state["symptom"] = symptom
  return state

Step 5. Node to classify the symptom

In [6]:
def classify_symptom(state: dict) ->dict:
  prompt=(
      "You are a helpful Medical Assistant, Classify the symptoms below into one of the categories \n"
      "-General\n -Emergency\n -Mental Health\n"
      f"symptom: {state['symptom']} \n"
      "Respond only with one word: General, Emergency or Mental Health"
      "Example: input: I have fever, output: General"
  )
  response = llm.invoke([HumanMessage(content=prompt)])
  category = response.content.strip()
  print(f"LLM classifies the symptom as: {category}")
  state["category"] = category
  return state

Step 6. Router Logic to route to the correct node

In [7]:
def symptom_router(state: dict) -> dict:
  cat=state["category"].lower()
  if "general" in cat:
    return "general_treatment"
  elif "emergency" in cat:
    return "emergency_treatment"
  elif "mental health" in cat:
    return "mental_health_treatment"
  else:
    return "general"

Step 7. Category specific response nodes

In [13]:
def general_node(state: dict)->dict:
  state["answer"] = f"'{state['symptom']}' : seems general: directing you to general ward for consulting a doctor"
  return state

def emergency_node(state: dict)->dict:
  state["answer"] = f"'{state['symptom']}' : It is medical emergency: seeking immediate help"
  return state

def mental_health_node(state: dict)->dict:
  state["answer"] = f"'{state['symptom']}' : seems like a mental health issue: talk to our councellor"
  return state

Step 8. Build LangGraph

In [15]:
builder= StateGraph(dict)

builder.set_entry_point("get_symptom")
builder.add_node("get_symptom", get_symptom)
builder.add_node("classify_symptom", classify_symptom)
builder.add_node("general_treatment", general_node)
builder.add_node("emergency_treatment", emergency_node)
builder.add_node("mental_health_treatment", mental_health_node)

builder.add_edge("get_symptom", "classify_symptom")
builder.add_conditional_edges("classify_symptom", symptom_router, {
    "general_treatment": "general_treatment",
    "emergency_treatment": "emergency_treatment",
    "mental_health_treatment": "mental_health_treatment"
})
builder.add_edge("general_treatment",END)
builder.add_edge("emergency_treatment",END)
builder.add_edge("mental_health_treatment",END)

Step 9. Compile and invoke the graph

In [17]:
graph=builder.compile()
final_state=graph.invoke({})
print("final Output \n")
print(final_state["answer"])

Please enter your symptomi have depression
LLM classifies the symptom as: Mental Health
final Output 

'i have depression' : seems like a mental health issue: talk to our councellor
